In [157]:
from sympy import Matrix, pprint

def game_price_limits(A):
    m, n = A.shape
    v_upper = min(max(A[i, :]) for i in range(m))
    v_lower = max(min(A[:, j]) for j in range(n))
    return v_lower, v_upper


def simplex_method(A, b, c, basis):
    m, n = A.shape

    # compute delta_k
    deltas = [
        c[j] - sum(c[basis[i]]*A[i, j] for i in range(m))
        for j in range(n)
    ]
    
    # check if optimal vertex is reached
    if all(delta >= 0 for delta in deltas):
        xstar = [0] * n
        for i in range(m):
            xstar[basis[i]] = b[i]
        return xstar

    k = deltas.index(min(deltas))  # determine in_index

    # check if target function is unbounded
    if all(A[i, k] <= 0 for i in range(m)): return

    # compute theta_k
    thetas = [
        b[i]/A[i, k] if A[i, k] > 0 else float('inf')
        for i in range(m)
    ]

    l = thetas.index(min(thetas))  # determine out_index
    basis[l] = k                   # change basis

    # perform gauss transformations
    b[l] /= A[l, k]
    A[l, :] /= A[l, k]
    for i in range(m):
        if i == l: continue
        b[i] -= b[l] * A[i, k]
        A[i, :] -= A[l, :] * A[i, k]

    return simplex_method(A, b, c, basis)   # start over



def solve_game(A): # accepts matrix for p2
    v_lower, v_upper = game_price_limits(A)
    print(f"v_lower = {v_lower}")
    print(f"v_upper = {v_upper}")
    if v_lower == v_upper:
        print("Game is solvable in pure optimal strategies")
        print(f'v* = {v_lower}')
        return
    
    m, n = A.shape
    c = -Matrix([1] * m + [0] * n)
    A = A.T.row_join(Matrix.eye(n))
    B = A.copy()
    b = Matrix([1] * n)
    basis = list(range(m, m+n))
    
    # find x_star using simplex method
    z = simplex_method(A, b, c, basis)
    v_star = 1/sum(z[:m])
    x_star = [v_star * z_i for z_i in z[:m]]
    
    # find y_star using dual simplex method
    D = Matrix([list(B[:, i]) for i in basis]).T.inv()
    C = -Matrix([[c[i, 0] for i in basis]])
    z = C*D
    y_star = [v_star * z_i for z_i in z[:n]]
    
    print(f'v* = {v_star}')
    print(f'x* = {x_star}')
    print(f'y* = {y_star}')


In [158]:
A = Matrix([
    [3, -1],
    [1, 2],
    [0, 4],
    [2, 1],
    [-1, 7],
    [1, 5]
])

solve_game(A)

v_lower = -1
v_upper = 2
v* = 7/5
x* = [1/5, 4/5, 0, 0, 0, 0]
y* = [3/5, 2/5]


In [159]:
A = Matrix([
    [0, 2],
    [5, 1]
])

solve_game(A)

v_lower = 1
v_upper = 2
v* = 5/3
x* = [2/3, 1/3]
y* = [1/6, 5/6]


In [161]:
A = Matrix([
    [1, 2, -5, 7],
    [9, 3, -2, 4],
    [6, 8, 10, -2],
    [0, 4, -3, -5]
])

v_lower, v_upper = game_price_limits(A)
print(f"v_lower = {v_lower}")
print(f"v_upper = {v_upper}")

assert v_lower > 0
assert v_upper > 0
print('v* > 0')

v_lower = 2
v_upper = 4
v* > 0
